In [1]:
import pyspark
import os
from dotenv import load_dotenv
from pathlib import Path

In [2]:
dotenv_path = Path("/opt/app/.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
spark_hostname = os.getenv("SPARK_MASTER_HOST_NAME")
spark_port = os.getenv("SPARK_MASTER_PORT")
kafka_host = os.getenv("KAFKA_HOST")
kafka_topic = os.getenv("KAFKA_TOPIC_NAME")

In [4]:
spark_host = f"spark://{spark_hostname}:{spark_port}"

In [5]:
os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2 org.postgresql:postgresql:42.2.18"


In [6]:
sparkcontext = pyspark.SparkContext.getOrCreate(
    conf=(pyspark.SparkConf().setAppName("DibimbingStreaming").setMaster(spark_host))
)
sparkcontext.setLogLevel("WARN")

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
stream_df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", f"{kafka_host}:9092")
    .option("subscribe", kafka_topic)
    .option("startingOffsets", "latest")
    .load()
)

(
    stream_df.selectExpr("CAST(value AS STRING)")
    .writeStream.format("console")
    .outputMode("append")
    .start()
    .awaitTermination()
)
